In [5]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [6]:
url = "http://www.espn.com/nba/salaries/_/year/2020"
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")

header = soup.find("tr", attrs={"class": "colhead"})
columns = [col.get_text() for col in header.find_all("td")]
columns = columns[1:]

df_salaries = pd.DataFrame(columns=columns)
df_salaries

,NAME,TEAM,SALARY


In [7]:
total_results = soup.find("div", attrs = {"class": "totalResults"})
num_players = int(total_results.get_text().split(" ")[0])
interval = 40

for a in range(1, num_players, interval):
    page_url = url + "/page/{}".format(int(a / interval)+1)
    page = requests.get(page_url)
    soup = BeautifulSoup(page.text, "html.parser")

    players = soup.find_all("tr", attrs = {"class":re.compile("row")})
    for player in players:
        stats = []
        i = 0
        for data in player.find_all("td"):
            string = data.get_text()
            if i == 1:
                name = string.split(",")[0]
                stats.append(name)
            elif i == 2:
                team = string
                stats.append(team)
            elif i == 3:
                space = ""
                salary = int(space.join(string[1:].split(",")))
                stats.append(salary)
            i += 1

        temp_df = pd.DataFrame(stats).transpose()
        temp_df.columns = columns

        df_salaries = pd.concat([df_salaries, temp_df], ignore_index=True)

df_salaries

,NAME,TEAM,SALARY
0,Stephen Curry,Golden State Warriors,40231758
1,Russell Westbrook,Houston Rockets,38506482
2,Chris Paul,Oklahoma City Thunder,38506482
3,James Harden,Houston Rockets,38199000
4,Kevin Durant,Brooklyn Nets,38199000
...,...,...,...
523,Jimmer Fredette,Phoenix Suns,208509
524,Jontay Porter,Memphis Grizzlies,197933
525,Anthony Tolliver,Memphis Grizzlies,183115
526,Tyler Johnson,Phoenix Suns,183115


In [8]:
df_salaries.to_csv("../Data/salaries18-19.csv", index=False, sep=',', encoding='utf-8')

In [9]:
df_salaries = pd.read_csv("../Data/salaries18-19.csv")
final_df = pd.read_csv("../Data/stats18-19.csv")

In [10]:
final_df.insert(final_df.shape[1]-1, "SALARY", int(0))
# number of columns is also len(final_df.columns)

In [11]:
for ind in range(final_df.shape[0]):
    for row in range(df_salaries.shape[0]):
        if final_df.loc[ind, "NAME"] == df_salaries.loc[row, "NAME"]:
            final_df.loc[ind, "TEAM"] = df_salaries.loc[row, "TEAM"]
            final_df.loc[ind, "SALARY"] = df_salaries.loc[row, "SALARY"]
            break
final_df.head()

,NAME,TEAM,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,REB,AST,STL,BLK,TO,DD2,TD3,SALARY,PER
0,1Bradley Beal,WSH,SG,10,35.9,34.9,12.1,24.5,49.4,2.4,...,87.4,5.3,5.0,1.4,0.7,3.3,1,0,0,27.43
1,2Kevin Durant,BKN,PF,12,36.3,31.3,10.4,19.5,53.4,2.8,...,86.7,7.5,6.0,0.8,1.3,3.7,5,0,0,28.32
2,3Stephen Curry,GS,PG,15,34.3,28.2,9.1,20.3,45.1,4.1,...,93.5,5.5,6.1,1.3,0.1,3.5,4,0,0,22.82
3,4Damian Lillard,POR,PG,14,36.2,28.1,8.5,19.3,44.1,3.7,...,94.5,4.7,6.7,1.0,0.3,3.1,1,0,0,24.42
4,5Joel Embiid,PHI,C,13,31.3,27.3,8.7,15.5,55.9,1.2,...,85.0,11.3,2.8,1.3,1.3,3.2,10,0,0,30.86


In [12]:
final_df.to_csv("../Data/alldata18-19.csv", index=False, sep=',', encoding='utf-8')